In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

from matplotlib.colors import LinearSegmentedColormap

from scipy.stats import binom

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [2]:
omdf = pd.read_excel(rootdir_data+'Omikron.xlsx')
omdf['Dato'] = pd.to_datetime(omdf['Dato'])
omdf['Ratio'] = omdf['AntalOmikron']/omdf['AntalTest']
omdf['Perc'] = 100 * omdf['AntalOmikron']/omdf['AntalTest']


In [3]:
# Calculate error of data, assuming binomial distribution
mean, var, skew, kurt = binom.stats(omdf.AntalTest,omdf.Ratio,moments = 'mvsk')
omdf['mean'] = mean 
omdf['var'] = var 
omdf['err'] = np.sqrt(var)/omdf.AntalTest
display(omdf)


C:\ProgramData\Anaconda3\envs\main\lib\site-packages\scipy\stats\_discrete_distns.py:88: RuntimeWarning: divide by zero encountered in _binom_skewness
  g1 = _boost._binom_skewness(n, p)
C:\ProgramData\Anaconda3\envs\main\lib\site-packages\scipy\stats\_discrete_distns.py:90: RuntimeWarning: divide by zero encountered in _binom_kurtosis_excess
  g2 = _boost._binom_kurtosis_excess(n, p)


,Dato,AntalTest,AntalOmikron,Ratio,AntalTestTotal,AntalOmikronTotal,RatioTotal,OmikronKMA,OmikronTCDK,AntalOmikronOldManuelt,Perc,mean,var,err
0,2021-11-21,130,2,0.015385,3693,2,0.054157,0,2,NaN,1.538462,2.0,1.969231,0.010795
1,2021-11-22,197,0,0.000000,4514,0,0.000000,0,0,NaN,0.000000,0.0,0.000000,0.000000
2,2021-11-23,231,0,0.000000,4718,0,0.000000,0,0,NaN,0.000000,0.0,0.000000,0.000000
3,2021-11-24,3505,0,0.000000,4034,0,0.000000,0,0,NaN,0.000000,0.0,0.000000,0.000000
4,2021-11-25,3658,0,0.000000,4106,0,0.000000,0,0,NaN,0.000000,0.0,0.000000,0.000000
5,2021-11-26,3703,0,0.000000,4164,1,0.024015,1,0,NaN,0.000000,0.0,0.000000,0.000000
6,2021-11-27,3637,2,0.000550,3853,2,0.051908,0,2,3.0,0.054990,2.0,1.998900,0.000389
7,2021-11-28,3581,9,0.002513,3892,9,0.231244,0,9,11.0,0.251326,9.0,8.977381,0.000837
8,2021-11-29,4534,11,0.002426,5098,11,0.215771,0,11,12.0,0.242611,11.0,10.973313,0.000731
9,2021-11-30,4887,24,0.004911,5426,26,0.479174,2,24,24.0,0.491099,24.0,23.882136,0.001000


In [4]:

firstDate = np.datetime64('2021-10-01')

latestsubdir = list(os.walk(path_data))[0][1][-1]
latestdir = path_data + latestsubdir
latestdir

dfCase = pd.read_csv(latestdir+'\\Test_pos_over_time.csv',dtype='str',delimiter=';',encoding='latin1')

# dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')
# dfAge['Nyindlagte pr. 100.000 borgere'] = pd.to_numeric(dfAge['Nyindlagte pr. 100.000 borgere'].str.replace(',','.'))

# dfAge.tail(18)
dfCase = dfCase.iloc[:-2] 
dfCase['NewPositive'] = pd.to_numeric(dfCase['NewPositive'].str.replace('.',''))
dfCase['Tested'] = pd.to_numeric(dfCase['Tested'].str.replace('.',''))
dfCase['PosPct'] = pd.to_numeric(dfCase['PosPct'].str.replace(',','.'))
dfCase['Date'] = pd.to_datetime(dfCase['Date'])
dfCase.tail()

<ipython-input-4-b96e666e5382>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfCase['NewPositive'] = pd.to_numeric(dfCase['NewPositive'].str.replace('.',''))
<ipython-input-4-b96e666e5382>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfCase['Tested'] = pd.to_numeric(dfCase['Tested'].str.replace('.',''))


,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
704,2021-12-31,9799,64.823,15.1,6.295,71118,53.569.530
705,2022-01-01,8690,66.953,13.0,6.364,73317,53.642.847
706,2022-01-02,20008,138.313,14.5,14.486,152799,53.795.646
707,2022-01-03,25141,193.794,13.0,20.516,214310,54.009.956
708,2022-01-04,8743,66.381,13.2,7.159,73540,54.083.496


In [5]:
# Plot general case data
fig,ax1 = plt.subplots(tight_layout=True)

ax1.fill_between(dfCase.Date,dfCase.NewPositive,color='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:

omdfShort = omdf.iloc[:-2]
omdfShort

firstOmikronDate = omdfShort.Dato.iloc[0]
lastOmikronDate = omdfShort.Dato.iloc[-1]
# np.where(dfCase.Date == firstOmikronDate)[0][0]
df_withOm = dfCase[(dfCase.Date >= firstOmikronDate) & (dfCase.Date <= lastOmikronDate)]
df_withOm.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
704,2021-12-31,9799,64.823,15.1,6.295,71118,53.569.530
705,2022-01-01,8690,66.953,13.0,6.364,73317,53.642.847
706,2022-01-02,20008,138.313,14.5,14.486,152799,53.795.646
707,2022-01-03,25141,193.794,13.0,20.516,214310,54.009.956
708,2022-01-04,8743,66.381,13.2,7.159,73540,54.083.496


In [27]:
omdf.tail()
omdfShort.tail()
# omdf

,Dato,AntalTest,AntalOmikron,Ratio,AntalTestTotal,AntalOmikronTotal,RatioTotal,OmikronKMA,OmikronTCDK,AntalOmikronOldManuelt,Perc,mean,var,err
40,2021-12-31,431,396,0.918794,10288,459,4.461509,63,396,NaN,91.879350,396.0,32.157773,0.013157
41,2022-01-01,409,386,0.943765,9219,458,4.968001,72,386,NaN,94.376528,386.0,21.706601,0.011391
42,2022-01-02,6021,5729,0.951503,21490,5827,27.114937,98,5729,NaN,95.150307,5729.0,277.838897,0.002768
43,2022-01-03,2264,2169,0.958039,27715,2274,8.204943,105,2169,NaN,95.803887,2169.0,91.013693,0.004214
44,2022-01-04,129,120,0.930233,25827,206,0.797615,86,120,NaN,93.023256,120.0,8.372093,0.022430


In [35]:

# ax1.fill_between(df_withOm.Date,df_withOm.NewPositive.values - omdfShort.AntalOmikron,df_withOm.NewPositive.values,color='r')
len(df_withOm)
len(omdfShort)
df_withOm.NewPositive.values*omdfShort.Ratio.values

array([5.63076923e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.09678306e+00, 9.67355487e+00,
       1.22470225e+01, 2.63720074e+01, 8.48411633e+01, 6.43717917e+01,
       8.19982706e+01, 1.14419873e+02, 1.68575778e+02, 3.56152587e+02,
       5.52827691e+02, 7.03941902e+02, 7.65088753e+02, 9.04301320e+02,
       1.14056290e+03, 1.63400722e+03, 3.06189114e+03, 4.41603907e+03,
       5.11246207e+03, 4.87967817e+03, 5.72479983e+03, 5.76809464e+03,
       6.47235462e+03, 8.56660525e+03, 1.00147478e+04, 9.21148195e+03,
       1.00843139e+04, 5.95775738e+03, 6.23869783e+03, 9.58232005e+03,
       2.03130327e+04, 1.99196644e+04, 1.60421538e+04, 1.75375831e+04,
       9.00325754e+03, 8.20132029e+03, 1.90376735e+04, 2.40860552e+04,
       8.13302326e+03])

In [55]:
# fig,ax1 = plt.subplots(tight_layout=True)
fig,(ax1,ax2) = plt.subplots(2,1,tight_layout=True,sharex=True)
ax1.fill_between(dfCase.Date,dfCase.NewPositive,color='gray')

curOmikronCount = df_withOm.NewPositive.values * omdfShort.Ratio.values

# ax1.plot(df_withOm.Date,df_withOm.NewPositive.values - omdf.AntalOmikron)
# ax1.fill_between(df_withOm.Date,df_withOm.NewPositive.values - omdfShort.AntalOmikron,df_withOm.NewPositive.values,color='r')
# ax1.fill_between(df_withOm.Date,df_withOm.NewPositive.values - df_withOm.NewPositive.values * omdfShort.Ratio.values,df_withOm.NewPositive.values,color='r')
ax1.fill_between(df_withOm.Date,df_withOm.NewPositive.values - curOmikronCount,df_withOm.NewPositive.values,color='r')
# ax1.fill_between(df_withOm.Date,df_withOm.NewPositive.values * omdfShort.Ratio.values,color='r')
# ax1.fill_between(omdf.Dato,omdf.AntalOmikron,color='r')
# ax1.fill_between(omdf.Dato,omdf.AntalOmikron,color='b')

ax2.fill_between(dfCase.Date,100*np.ones(dfCase.Date.shape),color='gray')
# ax2.fill_between(df_withOm.Date,100*(1-np.divide(df_withOm.NewPositive.values - omdfShort.AntalOmikron,df_withOm.NewPositive.values)),color='r')
ax2.fill_between(df_withOm.Date,100*np.ones(df_withOm.Date.shape),100-100*(np.divide(curOmikronCount,df_withOm.NewPositive.values)),color='r')


ax1.set_xlim(left=np.datetime64('2021-11-10'),right=omdfShort.Dato.max())
ax2.set_ylim([0,100])
# omdf
ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b\n%Y'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
fig,ax1 = plt.subplots(tight_layout=True)

ax1.plot(dfCase.Date,dfCase.NewPositive,'.:',color='k',linewidth=0.5)
ax1.fill_between(rnTime(dfCase.Date.values,7),rnMean(dfCase.NewPositive.values,7),color='gray')

ax1.fill_between(rnTime(df_withOm.Date.values,7),rnMean(df_withOm.NewPositive.values - omdfShort.AntalOmikron,7),rnMean(df_withOm.NewPositive.values,7),color='r')

ax1.set_xlim(left=np.datetime64('2021-10-10'),right=omdfShort.Dato.max()-np.timedelta64(3,'D'))

ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b\n%Y'))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig,(ax1,ax2) = plt.subplots(2,1,tight_layout=True,sharex=True)

ax1.plot(dfCase.Date,dfCase.NewPositive,'.:',color='k',linewidth=0.5)
ax1.fill_between(rnTime(dfCase.Date.values,7),rnMean(dfCase.NewPositive.values,7),color='gray')

ax1.fill_between(rnTime(df_withOm.Date.values,7),rnMean(df_withOm.NewPositive.values - omdfShort.AntalOmikron,7),rnMean(df_withOm.NewPositive.values,7),color='r')

ax1.set_xlim(left=np.datetime64('2021-10-10'),right=omdfShort.Dato.max()-np.timedelta64(3,'D'))

ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b\n%Y'))

ax2.fill_between(dfCase.Date,100*np.ones(dfCase.Date.shape),color='gray')
ax2.fill_between(df_withOm.Date,100*(1-np.divide(df_withOm.NewPositive.values - omdfShort.AntalOmikron,df_withOm.NewPositive.values)),color='r')


ax1.set_xlim(left=np.datetime64('2021-11-10'),right=omdfShort.Dato.max())
ax2.set_ylim([0,100])
# omdf
ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b\n%Y'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …